## Nelakuditi Rahul Naga - AI20BTECH11029

In [2]:
import numpy as np
import pandas as pd
from itertools import combinations
from scipy.linalg import null_space
import IPython.utils.io as io

def Simplex_Algorithm_Without_Feasible_Point(A, b, c):
    m = A.shape[0]
    n = A.shape[1]
    z_feasible = None

    if (b >= 0).all(): # Find a feasible point for the given linear optimization problem
      z_feasible = np.zeros(n)
    else:
      A_n = np.hstack((A,np.ones((m,1))))
      A_n_last_row = np.zeros((1,n+1))
      A_n_last_row[:,-1] = -1
      A_n = np.vstack((A_n,A_n_last_row))
      b_n = np.append(b,-np.min(b))
      c_n = np.zeros(n + 1)
      c_n[-1] = 1
      z_n = np.zeros(n + 1)
      z_n[-1] = np.min(b)

      with io.capture_output():
        z_feasible = Simplex_Algorithm_With_Feasible_Point(A_n,b_n,c_n,z_n)

      if z_feasible is not None:
        if z_feasible[-1] < 0:
          print('The given linear program has no feasible point.')
          return None
      else:
        comb = combinations(np.arange(len(A)),n)
        for i in comb:
          ind = np.array(i)
          if np.linalg.det(A[ind]) != 0:
            z_feasible = np.linalg.inv(A[ind]) @ b[ind]
          else:
            continue
          if np.count_nonzero(np.abs(A @ z_feasible - b) < 1e-10) == n and (A @ z_feasible <= b).all():
            break

    iter = 0
    while iter < 100000: # Make the linear optimization problem non-degenerate
      iter += 1
      if iter < 10000:
        b_temp = b.astype(float)
        eps = []
        e = np.random.uniform(1e-8,1e-7)
        for i in range(len(b)):
          eps.append(e**(i+1))
        eps = np.array(eps)
        b_temp += eps
      else:
        b_temp = b.astype(float)
        eps = []
        e = np.random.uniform(0.1,1)
        for i in range(len(b)):
          eps.append(e**(i+1))
        eps = np.array(eps)
        b_temp += eps

      if (b_temp >= 0).all():
        z_feasible = np.zeros(n)
      else:
        comb = combinations(np.arange(len(A)),n)
        for i in comb:
          ind = np.array(i)
          if np.linalg.det(A[ind]) != 0:
            z_feasible = np.linalg.inv(A[ind]) @ b_temp[ind]
          else:
            continue
          if np.count_nonzero(np.abs(A @ z_feasible - b_temp) < 1e-10) == n and (A @ z_feasible <= b_temp).all():
            break

      if len(np.argwhere(np.abs(A @ z_feasible - b_temp) < 1e-10).reshape(-1)) == n:
        b = b_temp
        break

    z = Simplex_Algorithm_With_Feasible_Point(A,b,c,z_feasible[:n])

    return z

def Simplex_Algorithm_With_Feasible_Point(A, b, c, z):
    if (A @ z > b).any():
      print(f'The given initial point {z} is infeasible.')
      return None
    elif (A @ z <= b).all():
      iter = 1
      try: # Move to a vertex if the given feasible point is not a vertex (also move in opposite direction for unbounded linear program)
        if np.count_nonzero(np.abs(A @ z - b) < 1e-10) != A.shape[1]:
          comb = combinations(np.arange(len(A)), A.shape[1])
          for i in comb:
            ind = np.array(i)
            if np.linalg.det(A[ind]) != 0:
              z = np.linalg.inv(A[ind]) @ b[ind]
            else:
              continue
            if np.count_nonzero(np.abs(A @ z - b) < 1e-10) == A.shape[1] and (A @ z <= b).all():
              break
      except:
        alpha = 0.1
        iter = 0
        while np.count_nonzero(np.abs(A @ z - b) < 1e-10) == 0 and iter < 10000:
          z_new = z + alpha*c
          if (A @ z_new <= b).all():
            z = z_new
          else:
           alpha /= 5
          iter += 1

        alpha = -0.1
        while np.count_nonzero(np.abs(A @ z - b) < 1e-10) == 0:
          z_new = z + alpha*c
          if (A @ z_new <= b).all():
            z = z_new
          else:
           alpha /= 5

        alpha = 0.1
        iter = 0
        while np.linalg.matrix_rank(A[np.argwhere(np.abs(A @ z - b) < 1e-10).reshape(-1)]) != A.shape[1] and iter < 10000:
          A1 = A[np.argwhere(np.abs(A @ z - b) < 1e-10).reshape(-1)]
          u = null_space(A1)[:,0]
          z_new = z + alpha*u
          if (A @ z_new <= b).all() and (np.count_nonzero(np.abs(A @ z_new - b) < 1e-10) > np.count_nonzero(np.abs(A @ z - b) < 1e-10)):
            z = z_new
          else:
            alpha /= 5
          iter += 1

        alpha = -0.1
        while np.linalg.matrix_rank(A[np.argwhere(np.abs(A @ z - b) < 1e-10).reshape(-1)]) != A.shape[1]:
          A1 = A[np.argwhere(np.abs(A @ z - b) < 1e-10).reshape(-1)]
          u = null_space(A1)[:,0]
          z_new = z + alpha*u
          if (A @ z_new <= b).all() and (np.count_nonzero(np.abs(A @ z_new - b) < 1e-10) > np.count_nonzero(np.abs(A @ z - b) < 1e-10)):
            z = z_new
          else:
            alpha /= 5

      print(f'z{iter} = {z}, C{iter} = {c @ z}')

      while True: # Jump from one vertex to another until the optimal vertex is reached
        A1 = A[np.argwhere(np.abs(A @ z - b) < 1e-10).reshape(-1)]
        A1_inv = np.linalg.inv(A1)

        if len(np.argwhere(A1_inv.T @ c < 0).reshape(-1)) == 0:
          break
        else:
          p = -A1_inv[:,np.argwhere(A1_inv.T @ c < 0).reshape(-1)[0]]

          if len(np.argwhere(A @ p > 0).reshape(-1)) == 0:
            print('The given linear program is unbounded.')
            return None

          A2 = A[np.array([ind for ind in np.arange(len(A)) if ind not in np.argwhere(np.abs(A @ z - b) < 1e-10).reshape(-1)])]
          b2 = b[np.array([ind for ind in np.arange(len(A)) if ind not in np.argwhere(np.abs(A @ z - b) < 1e-10).reshape(-1)])]

          step_sizes = ((b2 - (A2 @ z))[(A2 @ p) != 0])/((A2 @ p)[(A2 @ p) != 0])
          try:
            ε = np.min((step_sizes)[(step_sizes) >= 0])
          except:
            print('The given linear program is unbounded.')
            return None

          z =  z + ε*p

        iter += 1
        print(f'z{iter} = {z}, C{iter} = {c @ z}')
    return z

# Reading the input data from a csv file
data_df = pd.read_csv('data.csv',header = None) # Modify the data.csv file accordingly for new input data
data_array = data_df.to_numpy(dtype = float)
m = data_array.shape[0] - 1
n = data_array.shape[1] - 1
A = data_array[1:][:,:-1]
b = data_array[:,-1][1:]
c = data_array[0][:-1]
#print(A,b,c)

x = Simplex_Algorithm_Without_Feasible_Point(A, b, c)
if x is not None:
  print(f'The solution is given by x = {x} and the optimal value is {c @ x}.')

z1 = [2.00000000e+00 2.31081039e-08], C1 = -1.0000000231081039
z2 = [2. 0.], C2 = -1.0
z3 = [1. 0.], C3 = -0.4999999999999998
The solution is given by x = [1. 0.] and the optimal value is -0.4999999999999998.
